**필요 모듈 호출**

In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

---

# Pivoting

- 가지고 있는 데이터원본을 원하는 형태의 가공된 정보를 보여주는 것
    - 자료의 형태를 변경하기 위해 많이 사용하는 방법
    
![image](https://user-images.githubusercontent.com/75322297/149518966-713629cd-a050-4e1e-ae02-aec91d446644.png)

- 출처 : https://rfriend.tistory.com/275

## Pivoting Method

### 1) pivot( )
- pandas.pivot(data, index=None, columns=None, values=None)
- DataFrame.pivot(index=None, columns=None, values=None)

#### pivot() 사용예제

In [3]:
# 데이터 생성
data = {
    "도시": ["서울", "서울", "서울", "부산", 
           "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015",
           "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737,
           3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권",
           "경상권", "경상권", "수도권", "수도권"]
}

columns = ["도시", "연도", "인구", "지역"]

df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [5]:
# 각 도시에 대한 연도별 평균 인구
df1.pivot('도시', '연도', '인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [6]:
# 각 지역별 도시에 대한 연도별 평균 인구
df1.pivot(['지역','도시'], '연도', '인구')

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

<br>

### 2) pivot_table( )

- **pivot_table(data, index=None, columns=None, values=None**,
    aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)
    1. index :  **행 인덱스**로 들어갈 키열 또는 키열의 리스트
    2. columns : **열 인덱스**로 들어갈 키열 또는 키열의 리스트
    3. values : 분석할 데이터 프레임에서 분석할 열
    4. fill_value : NaN이 표출될 때 대체값 지정
    5. margins : 모든 데이터를 분석한 결과를 행으로 표출할 지 여부
    6. margins_name : margins가 표출될 때 그 열(행)의 이름


- 피봇테이블을 작성할 때 반드시 설정해야 되는 인수
    - data : 사용 데이터 프레임
    - index : 행 인덱스로 사용할 필드(기준 필드로 작용됨)
    - values : 인덱스 명을 제외한 나머지 값(data)은 수치 data 만 사용함
    - aggfunc: 기본 함수가 평균(mean)함수 이기 때문에 각 데이터의 평균값이 반환

#### pivot_table() 사용 예제

In [13]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')[['age','sex','class','fare','survived']]
df.head()


,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [16]:
# 선실 등급별 숙박객의 생존자 수
pd.pivot_table(df, index='class', columns='survived', values='sex', aggfunc='size')

survived,0,1
class,,
First,80,136
Second,97,87
Third,372,119


In [17]:
# 각 선실 등급별 숙박객의 생존자 수와 평균 나이
pd.pivot_table(df, index='class', columns='survived',
               values='age', aggfunc='mean')

survived,0,1
class,,
First,43.695312,35.368197
Second,33.544444,25.901566
Third,26.555556,20.646118


In [19]:
# 생존자/사망자의 최소, 최대 나이(성별별)
pd.pivot_table(df, index=['survived'], columns='sex',
               values=['age'], aggfunc=['min','max'])

min          max      
            age          age      
sex      female  male female  male
survived                          
0          2.00  1.00   57.0  74.0
1          0.75  0.42   63.0  80.0

---

<br>

# Stacking & UnStacking

- 스태킹(stacking) : 컬럼 레이블과 그 값을 로우 인덱스와 값으로 회전시킴
- 언스태킹(unstacking) : 로우 인덱스와 그 값이 컬럼 레이블과 값으로 회전시킴

- DataFrame.unstack(level=- 1, fill_value=None)
    - level : int, str, list, default=-1
        - 언스태킹을 적용하는 레벨
        - 기본값은 마지막 레벨 : 언스태킹 결과 항상 마지막 레벨로 이동
    - fill_value : int, str or dict
        - 언스태킹 결과 결측치는 NaN으로 대체
        
        
- DataFrame.stack(level=- 1, dropna=True)
    - level : int, str, list, default=-1
        - 언스태킹을 적용하는 레벨
        - 기본값은 마지막 레벨 : 언스태킹 결과 항상 마지막 레벨로 이동
    - dropna : bool, default True
        - 스태킹 결과 결측치 처리 여부, 기본값은 True로 결측치 제외

![](./img/stacking.png)

## 1) Stacking

### 1. single level columns

In [20]:
# single level columns
df = pd.DataFrame([[0, 1], [2, 3]],
                  index=['cat', 'dog'],
                  columns=['weight', 'height'])
df

,weight,height
cat,0,1
dog,2,3


**`⬇stacking⬇`**

In [11]:
df.stack()

cat  weight    0
     height    1
dog  weight    2
     height    3
dtype: int64

### 2. multi level columns1

In [21]:
multicol = pd.MultiIndex.from_tuples([('weight','kg'),
                                      ('weight','pounds')])
df2 = pd.DataFrame([[0, 1], [2, 3]],
                  index=['cat', 'dog'],
                  columns=multicol)
df2


weight       
        kg pounds
cat      0      1
dog      2      3

**`⬇stacking⬇`**

In [22]:
# 마지막 레벨로 이동
df2.stack()

weight
cat kg           0
    pounds       1
dog kg           2
    pounds       3

### 3. multi-level columns2

In [29]:
multicol2 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('height', 'm')])
df3 = pd.DataFrame([[None, 1.0], [2.0, 3.0]],
                     index=['cat', 'dog'],
                     columns=multicol2)
df3

,weight,height
,kg,m
cat,NaN,1.0
dog,2.0,3.0


**`⬇stacking⬇`**

In [30]:
# 스태킹 결과 마지막 레벨로 이동
df3.stack()

# 스태킹 결과 첫번째 레벨로 이동
df3.stack(0)

# 두 단계 레벨 모두 이동
df3.stack([0,1])

height  weight
cat m      1.0     NaN
dog kg     NaN     2.0
    m      3.0     NaN

kg    m
cat height  NaN  1.0
dog height  NaN  3.0
    weight  2.0  NaN

cat  height  m     1.0
dog  height  m     3.0
     weight  kg    2.0
dtype: float64

In [31]:
# 스태킹 결과 결측치 행 포함
df3.stack(dropna=False)

height  weight
cat kg     NaN     NaN
    m      1.0     NaN
dog kg     NaN     2.0
    m      3.0     NaN

In [32]:
# 스태킹 결과 결측치 행 삭제
df3.stack(dropna=True)

height  weight
cat m      1.0     NaN
dog kg     NaN     2.0
    m      3.0     NaN

---

## 2. Unstacking

In [33]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
                                   ('two', 'a'), ('two', 'b')])
s = pd.Series(np.arange(1.0, 5.0), index=index)
s


one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

**`⬇unstacking⬇`**

In [35]:
s.unstack() #default = level = -1
s.unstack(level=0) # 첫번째 레벨로

,a,b
one,1.0,2.0
two,3.0,4.0


,one,two
a,1.0,3.0
b,2.0,4.0


---

# Group Analysis

- 키에 의해서 결정되는 데이터가 여러 개 있는 경우 미리 지정한 연산을 통해 각 그룹 데이터의 대표값을 계산하는 것


- 만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석을 해야 함

## 패턴

- 분할(SPLIT) : 어떤 기준에 따라 데이터셋을 작은 조각으로 분할한다
- 적용(APPLY) : 각 조각에 대해 독립적으로 연산을 수행한다
- 조합(COMBINE) : 모든 결과를 다시 하나의 단위로 조합한다

![](./img/groupby.png)

## 데이터 분할(split) 작업
- 판다스의 시리즈, 데이터프레임을 **groupby() 메소드**로 수행하며 그룹 객체에 대해 그룹연산을 수행
- 하나 이상의 **인덱스 레이블**이나 **컬럼(컬럼 리스트)**을 지정하여 이에 기초하여 연관된 값들이 그룹화 됨
- **GroupBy 클래스 객체 반환**
- **그룹 연산 메서드 사용 가능**
![image](https://user-images.githubusercontent.com/75322297/149522059-8b40b132-57b0-46e8-ba12-261a5d961807.png)

- 사진 출처 : https://pandas.pydata.org/docs/user_guide/groupby.html#aggregation

## 데이터 분할 후 작업

- 집계(aggregation) : 각 그룹의 아이템에 대한 평균이나 개수 계산
- 변형(transform) : 그룹이나 아이템에 특정적인 계산을 수행
- 필터링(filtering) : 그룹 단위의 계산에 기초해 불필요한 데이터 그룹 제거

---

<br>

![](./img/groupby2.png)

## 1. 데이터 분할

1. 단일 컬럼의 그룹화
2. 복수 컬럼의 그룹화
3. 인덱스 레벨을 이용한 그룹화

### 예제 데이터 준비

In [36]:
data=pd.DataFrame({
    'key1':['A','A','B','B','A'] , 
    'key2': ['one', 'two', 'one', 'two' , 'one'],
    'data1':[1,2,3,4,5],
    'data2':[10,20,30,40,50]})
data


,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


### 1) 단일 컬럼의 그룹화

In [38]:
gr = data.groupby(data.key1) # key1 하나로
gr

**groups 속성 : GroupBy object**
- 각 그룹의 이름을 키로 갖는 파이썬 딕셔너리를 반환
- 딕셔너리 값은 각 그룹에 속하는 인덱스 레이블의 배열
- Dict {group name -> group labels}

In [39]:
gr.groups
gr.ngroups # ngroups는 groups의 수를 반환한다.

{'A': [0, 1, 4], 'B': [2, 3]}

2

**그룹화된 결과 접근**

In [40]:
# 그룹 내용 출력함수 정의 
def print_groups(group_object):
    for name, group in group_object:
        print(name)
        print(group[:5])


In [41]:
print_groups(gr)

A
  key1 key2  data1  data2
0    A  one      1     10
1    A  two      2     20
4    A  one      5     50
B
  key1 key2  data1  data2
2    B  one      3     30
3    B  two      4     40


-> 각 그룹은 그룹명과 일치하는 값들의 로우들로 구성된 데이터프레임을 하나씩 포함하고 있음

**그룹정보 groups를 데이터프레임으로 저장**

In [44]:
grDF = pd.DataFrame(gr)
grDF

,0,1
0,A,key1 key2 data1 data2 0 A one 1 ...
1,B,key1 key2 data1 data2 2 B one 3 ...


- 첫번째 그룹 정보

In [47]:
grDF.loc[0]
grDF.loc[0].values

0                                                    A
1      key1 key2  data1  data2
0    A  one      1  ...
Name: 0, dtype: object

array(['A',
         key1 key2  data1  data2
0    A  one      1     10
1    A  two      2     20
4    A  one      5     50], dtype=object)

- 두번째 그룹 정보

In [48]:
grDF.loc[1]
grDF.loc[1].values

0                                                    B
1      key1 key2  data1  data2
2    B  one      3  ...
Name: 1, dtype: object

array(['B',
         key1 key2  data1  data2
2    B  one      3     30
3    B  two      4     40], dtype=object)

**메소드**

In [53]:
gr.size() # 각 그룹의 크기 반환
gr.count() # 각 그룹의 컬럼별 아이템 개수 반환
groups.sum() # 각 그룹의 컬럼별 합계 반환

key1
A    3
B    2
dtype: int64

,key2,data1,data2
key1,,,
A,3,3,3
B,2,2,2


,data1,data2
key1,,
A,8,80
B,7,70


**선택한 컬럼에 대한 그룹별 합계 반환1**
- 그룹객체[컬럼명].sum()
- 그룹객체.sum()[컬럼명]
- 반환값은 시리즈 형식

In [55]:
groups['data1'].sum() # groups.sum()['data1']

key1
A    8
B    7
Name: data1, dtype: int64

**선택한 컬럼에 대한 그룹별 합계 반환2**
- [컬럼명]리스트로 지정한 경우 : 데이터프레임 형식으로 반환
- 그룹객체[[컬럼명]].sum()
- 그룹객체.sum()[[컬럼명]]

In [56]:
groups[['data1']].sum() # groups.sum()[['data1']]

,data1
key1,
A,8
B,7


### 2) 복수 컬럼의 그룹화

- .groupby( ) 메소드에 그룹명의 리스트를 전달하면 여러 컬럼에 대해 그룹화를 수행

In [57]:
gr2 = data.groupby([data.key1, data.key2])
gr2

**그룹정보, 그룹수 확인**

In [61]:
gr2.groups # 복수 컬럼으로 된 그룹정보 : 튜플형태로 반환
gr2.ngroups

{('A', 'one'): [0, 4], ('A', 'two'): [1], ('B', 'one'): [2], ('B', 'two'): [3]}

4

**그룹정보 출력 : print_groups()함수 정의**

In [62]:
print_groups(gr2)

('A', 'one')
  key1 key2  data1  data2
0    A  one      1     10
4    A  one      5     50
('A', 'two')
  key1 key2  data1  data2
1    A  two      2     20
('B', 'one')
  key1 key2  data1  data2
2    B  one      3     30
('B', 'two')
  key1 key2  data1  data2
3    B  two      4     40


### 3) 인덱스 레벨을 이용한 그룹화

- 컬럼 대신 인덱스의 값을 사용해 그룹화

**예제 데이터**
- 계층형(멀티) 인덱스를 갖도록 데이터 구성 변경

In [63]:
data
data2 = data.copy()
data2 = data2.set_index(['key1','key2'])
data2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


data1  data2
key1 key2              
A    one       1     10
     two       2     20
B    one       3     30
     two       4     40
A    one       5     50

**레벨0(key1)을 사용해 그룹화**

In [64]:
print_groups(data2.groupby(level=0))

A
           data1  data2
key1 key2              
A    one       1     10
     two       2     20
     one       5     50
B
           data1  data2
key1 key2              
B    one       3     30
     two       4     40


**레벨1(key2)을 사용해 그룹화**

In [65]:
print_groups(data2.groupby(level=1))

one
           data1  data2
key1 key2              
A    one       1     10
B    one       3     30
A    one       5     50
two
           data1  data2
key1 key2              
A    two       2     20
B    two       4     40


**예제 데이터 : iris**

In [66]:
import seaborn as sns
iris = sns.load_dataset("iris")

In [74]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


**픔종별로 그룹화**

In [75]:
# iris 품종별로 그룹
igr = iris.groupby(iris.species)
igr

**품종별 합계 계산**

In [76]:
igr.sum()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


**품종별 평균 계산**

In [77]:
igr.mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


**품종별 기술통계**

In [79]:
igr.describe()

sepal_length                                              \
                  count   mean       std  min    25%  50%  75%  max   
species                                                               
setosa             50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8   
versicolor         50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0   
virginica          50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9   

           sepal_width         ... petal_length      petal_width         \
                 count   mean  ...          75%  max       count   mean   
species                        ...                                        
setosa            50.0  3.428  ...        1.575  1.9        50.0  0.246   
versicolor        50.0  2.770  ...        4.600  5.1        50.0  1.326   
virginica         50.0  2.974  ...        5.875  6.9        50.0  2.026   

                                               
                 std  min  25%  50%  75%  max  
species                                        
setosa      0.105386  0.1  0.2  0.2  0.3  0.6  
versicolor  0.197753  1.0  1.2  1.3  1.5  1.8  
virginica   0.274650  1.4  1.8  2.0  2.3  2.5  

[3 rows x 32 columns]

### 4) [ ] 연산자를 사용해 특정 컬럼에 대한 집계 연산 수행

In [80]:
igr['sepal_length'].mean()

species
setosa        5.006
versicolor    5.936
virginica     6.588
Name: sepal_length, dtype: float64

In [81]:
igr[['sepal_length','petal_length']].mean()

,sepal_length,petal_length
species,,
setosa,5.006,1.462
versicolor,5.936,4.260
virginica,6.588,5.552


In [82]:
igr['petal_length'].describe()

,count,mean,std,min,25%,50%,75%,max
species,,,,,,,,
setosa,50.0,1.462,0.173664,1.0,1.4,1.50,1.575,1.9
versicolor,50.0,4.260,0.469911,3.0,4.0,4.35,4.600,5.1
virginica,50.0,5.552,0.551895,4.5,5.1,5.55,5.875,6.9


---

## 2-1 그룹별 집계 함수

**GroupBy.그룹함수(func, *args, **kwargs)**

- func : 함수이름
    - 내장함수는 np.sum 또는 'sum'으로 지정, 사용자정의함수는 함수이름 그대로
- *args : 함수의 매개변수들
- **kwargs : 함수에 들어가는 키워드 인수들

### 1) agg(), aggregate()

#### 각 컬럼의 최대값과 최소값의 비율 계산
- 함수 peak_to_peak_ratio() 작성

In [88]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()
 # 함수 반환 값이 수치 스칼라 타입
 # 최대값과 최소값의 비율 계산

In [89]:
# 품종별로 사용자 정의함수 peak_to_peak_ratio 연산 적용
igr.agg(peak_to_peak_ratio)


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


#### agg() 메소드에 여러 함수 적용

**.agg([함수1, 함수2,...])**

In [86]:
igr.agg([np.sum, np.mean])

sepal_length        sepal_width        petal_length         \
                    sum   mean         sum   mean          sum   mean   
species                                                                 
setosa            250.3  5.006       171.4  3.428         73.1  1.462   
versicolor        296.8  5.936       138.5  2.770        213.0  4.260   
virginica         329.4  6.588       148.7  2.974        277.6  5.552   

           petal_width         
                   sum   mean  
species                        
setosa            12.3  0.246  
versicolor        66.3  1.326  
virginica        101.3  2.026

In [91]:
igr.agg(['sum', 'mean'])

sepal_length        sepal_width        petal_length         \
                    sum   mean         sum   mean          sum   mean   
species                                                                 
setosa            250.3  5.006       171.4  3.428         73.1  1.462   
versicolor        296.8  5.936       138.5  2.770        213.0  4.260   
virginica         329.4  6.588       148.7  2.974        277.6  5.552   

           petal_width         
                   sum   mean  
species                        
setosa            12.3  0.246  
versicolor        66.3  1.326  
virginica        101.3  2.026

### 2) apply( )

: 어떤 함수나 적용 가능

**Top3 산출**
- petal_length 기준으로 Top3 항목 선택

In [92]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3] 

# 함수 반환값이 수치 집합

- 품종별로 petal_length 기준의 Top3

In [93]:
iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

- agg( )는 반환값이 수치스칼라인 경우만 사용 가능
- apply()는 반환값이 수치 집합인 경우 적용 가능

In [65]:
# iris.groupby(iris.species).agg(top3_petal_length) 
# 반환값이 수치 스칼라인 경우 적용 가능한데 반환값이 수치 집합 이므로 에러 발생

- 품종별로 최대값과 최소값 비중

In [94]:
iris.groupby(iris.species).apply(peak_to_peak_ratio) 

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,1.348837,1.913043,1.900000,6.000000
versicolor,1.428571,1.700000,1.700000,1.800000
virginica,1.612245,1.727273,1.533333,1.785714


**수치형자료를 대,중,소 3범주를 갖는 범주형으로 변환 : qcut()**

In [95]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"]).astype(str)

**품종별로 petal_length를 대,중,소 3범주로 변환**

In [96]:
iris.groupby(iris.species).petal_length.apply(q3cut)

0      소
1      소
2      소
3      중
4      소
      ..
145    소
146    소
147    소
148    중
149    소
Name: petal_length, Length: 150, dtype: object

**품종별로 petal_length를 대,중,소 3범주로 변환된 값을 petal_length_class변수로 추가** 

In [97]:
iris["petal_length_class"] = iris.groupby(iris.species).petal_length.apply(q3cut)
iris.head(10)
iris.tail(10)

,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_class
0,5.1,3.5,1.4,0.2,setosa,소
1,4.9,3.0,1.4,0.2,setosa,소
2,4.7,3.2,1.3,0.2,setosa,소
3,4.6,3.1,1.5,0.2,setosa,중
4,5.0,3.6,1.4,0.2,setosa,소
5,5.4,3.9,1.7,0.4,setosa,대
6,4.6,3.4,1.4,0.3,setosa,소
7,5.0,3.4,1.5,0.2,setosa,중
8,4.4,2.9,1.4,0.2,setosa,소
9,4.9,3.1,1.5,0.1,setosa,중


,sepal_length,sepal_width,petal_length,petal_width,species,petal_length_class
140,6.7,3.1,5.6,2.4,virginica,중
141,6.9,3.1,5.1,2.3,virginica,소
142,5.8,2.7,5.1,1.9,virginica,소
143,6.8,3.2,5.9,2.3,virginica,대
144,6.7,3.3,5.7,2.5,virginica,중
145,6.7,3.0,5.2,2.3,virginica,소
146,6.3,2.5,5.0,1.9,virginica,소
147,6.5,3.0,5.2,2.0,virginica,소
148,6.2,3.4,5.4,2.3,virginica,중
149,5.9,3.0,5.1,1.8,virginica,소


----------------------------------------

## 2-2 데이터 그룹의 변형 : transform()

### 데이터 변형의 일반적인 과정

: transform() 메소드는 데이터프레임의 **모든 값**에 함수를 적용하며, 다음의 특성을 갖는 새 데이터프레임을 반환한다

- 모든 그룹의 인덱스가 합쳐진 인덱스를 갖는다
- row의 개수는 모든 그룹의 row 개수의 총합과 같다
- 그룹화 대상이 아닌 컬럼도 함수가 성공적으로 적용된다면 결과에 포함되며, 그렇지 않은 컬럼은 삭제될 수 있다

**예제 데이터1**

In [98]:
trans_data = pd.DataFrame({'Label' : ['A','C','B','A','C'],
                           'Values': [0, 1, 2, 3, 4],
                           'Values2' : [5, 6, 7, 8, 9],
                           'Other': ['foo','bar','baz','fiz','buz']},
                         index=list('VWXYZ'))
trans_data

,Label,Values,Values2,Other
V,A,0,5,foo
W,C,1,6,bar
X,B,2,7,baz
Y,A,3,8,fiz
Z,C,4,9,buz


**Label 컬럼으로 그룹화**

In [99]:
groups = trans_data.groupby('Label')
groups

In [100]:
print_groups(groups)

A
  Label  Values  Values2 Other
V     A       0        5   foo
Y     A       3        8   fiz
B
  Label  Values  Values2 Other
X     B       2        7   baz
C
  Label  Values  Values2 Other
W     C       1        6   bar
Z     C       4        9   buz


In [101]:
groups.groups

{'A': ['V', 'Y'], 'B': ['X'], 'C': ['W', 'Z']}

### 각 값에 10을 더하는 함수를 적용해 데이터프레임 변형 수행

In [102]:
groups.transform(lambda x:x+10)

,Values,Values2
V,10,15
W,11,16
X,12,17
Y,13,18
Z,14,19


-> 'Label'과 'Other' 컬럼은 문자열 값들이므로 함수 적용에 실패하여 두 컬럼은 결과에서 제외됨

### 그룹의 평균으로 결측값 채우기

**예제 데이터2**

In [103]:
trans_data2 = pd.DataFrame({'Label' : list('ABABAB'),
                            'Values' : [10, 20, 11, np.nan, 12, 22]})
trans_data2

,Label,Values
0,A,10.0
1,B,20.0
2,A,11.0
3,B,NaN
4,A,12.0
5,B,22.0


In [106]:
groups= trans_data2.groupby('Label')
groups.groups
print_groups(groups)

{'A': [0, 2, 4], 'B': [1, 3, 5]}

A
  Label  Values
0     A    10.0
2     A    11.0
4     A    12.0
B
  Label  Values
1     B    20.0
3     B     NaN
5     B    22.0


**각 그룹의 평균 계산**

In [107]:
groups.mean()

,Values
Label,
A,11.0
B,21.0


**결측치(Nan) 채우기**

- 그룹별 평균으로 채우기

In [108]:
fillna = groups.transform(lambda x : x.fillna(x.mean()))
fillna

,Values
0,10.0
1,20.0
2,11.0
3,21.0
4,12.0
5,22.0


## 2-3 그룹 필터링 : filter()

- 데이터그룹을 선택적으로 삭제

**예제 데이터3**

In [109]:
df3 = pd.DataFrame({'Label' : list('AABCCC'),
                    'Values' : [1, 2, 3, 4, np.nan, 8]})
df3

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0
3,C,4.0
4,C,NaN
5,C,8.0


**아이템의 개수가 지정한 최소한의 수를 넘지 않는 그룹을 제외**

In [110]:
df3.groupby('Label').filter(lambda x:x.Values.count() > 1)

,Label,Values
0,A,1.0
1,A,2.0
3,C,4.0
4,C,NaN
5,C,8.0


**NaN이 하나라도 존재하는 그룹 제외**

In [111]:
df3.groupby('Label').filter(lambda x:x.Values.isnull().sum() == 0)

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0


**전체 데이터셋 평균과의 차이가 2.0이 넘는 그룹 평균을 가진 그룹만 필터링**

In [112]:
df3.groupby('Label').mean()

,Values
Label,
A,1.5
B,3.0
C,6.0


In [113]:
gmean = df3.groupby('Label').mean().mean()

In [115]:
df3.groupby('Label').filter(lambda x: (x.Values.mean()- gmean) > 2.0)

,Label,Values
3,C,4.0
4,C,NaN
5,C,8.0
